In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

rd_velomagg = pd.read_csv("../data/video/courses/TAM_MMM_CoursesVelomagg.csv", encoding="utf-8-sig")


def correct_encoding(value):
    if isinstance(value, str):
        return value.encode("latin1", errors="replace").decode("utf-8", errors="replace")
    return value

rd_velomagg = rd_velomagg.apply(lambda col: col.apply(correct_encoding) if col.dtype == "object" else col)

rd_velomagg["Date"] = pd.to_datetime(rd_velomagg["Departure"]).dt.date

date_debut = pd.to_datetime("2024-03-18").date()
date_fin = pd.to_datetime("2024-03-24").date()

rd_velomagg = rd_velomagg[(rd_velomagg["Date"] >= date_debut) & (rd_velomagg["Date"] <= date_fin)]
dates = pd.date_range(date_debut, date_fin).date
weekdays = {date.strftime("%A"): date for date in dates}
def plot_interactive_graph(selected_day="lundi", show="both"):
    fig = make_subplots(rows=1, cols=1)

    for idx, (jour, date) in enumerate(weekdays.items()):
        rd_velo_par_jour = rd_velomagg[rd_velomagg["Date"] == date]
        
        r_sum = rd_velo_par_jour["Departure station"].value_counts()
        d_sum = rd_velo_par_jour["Return station"].value_counts()

        stations = sorted(set(r_sum.index).union(set(d_sum.index)))
        r_sum = r_sum.reindex(stations, fill_value=0)
        d_sum = d_sum.reindex(stations, fill_value=0)

        fig.add_trace(go.Bar(
            x=r_sum, 
            y=stations, 
            name=f"Retrait - {jour}", 
            marker=dict(color="plum"),
            visible=(jour == selected_day), 
            orientation="h"
        ))

        fig.add_trace(go.Bar(
            x=d_sum, 
            y=stations, 
            name=f"Dépôt - {jour}", 
            marker=dict(color="magenta"),
            visible=(jour == selected_day), 
            orientation="h"
        ))

    fig.update_layout(
        title="Retraits et Dépôts des Vélomaggs",
        xaxis_title="Nombre de retraits ou dépôts",
        yaxis_title="Stations",
        barmode='group', 
        yaxis_tickangle=-45,
        legend=dict(
             yanchor="bottom",
             y=0,
             xanchor="left",
             x=1
        ),
        bargap=0.05,   
        autosize=False,
        width=900,
        height=1000,   
        template="seaborn",
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": ["visible", [True] * len(weekdays) * 2],
                        "label": "Tous les jours",
                        "method": "restyle"
                    },
                    *[
                        {
                            "args": [
                                "visible", 
                                [i // 2 == idx for i in range(len(weekdays) * 2)]
                            ],
                            "label": jour,
                            "method": "restyle"
                        }
                        for idx, jour in enumerate(weekdays)
                    ],
                ],
                "direction": "down",
                "showactive": True,
                "x": 1,
                "xanchor": "left",
            }
        ], 
    )

    fig.write_html("../docs/graphique_interactif.html") 

plot_interactive_graph(selected_day="lundi", show="both")
